# Crust 1.0

In this notebook, we will implement [Crust 1.0](https://igppweb.ucsd.edu/~gabi/crust1.html) as four `AxiSEM3D` models:

1. A volumetric model for P-velocity, S-velocity and density (`StructuredGridV3D`)
2. A geometric model for surface undulation (`StructuredGridG3D`)
3. A geometric model for moho undulation (`StructuredGridG3D`)
4. An ocean-load model for the ocean (`StructuredGridO3D`)


In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from skimage.filters import gaussian
import matplotlib.pyplot as plt
from netCDF4 import Dataset

# 1. User options

Please specify the following options: 

In [ ]:
# whether to consider the ice layer in Crust 1.0
INCLUDE_ICE = False

# whether to consider the three sediment layers in Crust 1.0
INCLUDE_SEDIMENT = False

# radius of surface in the mesh
RADIUS_SURFACE = 6371e3

# radius of moho in the mesh
RADIUS_MOHO = 6346.6e3

# how many element layers between surface and moho in the mesh
# you can find this number by dragging the mesh file into Paraview
N_ELEM_LAYERS = 2

## The volumetric model

In [ ]:
nlayer = 9
nlat = 180
nlon = 360
nrow = nlat * nlon

# read raw data
bnd = np.loadtxt('crust1.0_raw_data/crust1.bnds')
v_p = np.loadtxt('crust1.0_raw_data/crust1.vp')
v_s = np.loadtxt('crust1.0_raw_data/crust1.vs')
rho = np.loadtxt('crust1.0_raw_data/crust1.rho')

# interpolate latitude from 0 to 180
mRl = np.zeros((nlat + 1, nlon, nlayer))
mVp = np.zeros((nlat + 1, nlon, nlayer))
mVs = np.zeros((nlat + 1, nlon, nlayer))
mRh = np.zeros((nlat + 1, nlon, nlayer))
for col in range(nlayer):
    # north pole
    mRl[0, :, col] = bnd[0:nlon, col].sum() / nlon
    mVp[0, :, col] = v_p[0:nlon, col].sum() / nlon
    mVs[0, :, col] = v_s[0:nlon, col].sum() / nlon
    mRh[0, :, col] = rho[0:nlon, col].sum() / nlon
    # south pole
    mRl[-1, :, col] = bnd[nrow - nlon:nrow, col].sum() / nlon
    mVp[-1, :, col] = v_p[nrow - nlon:nrow, col].sum() / nlon
    mVs[-1, :, col] = v_s[nrow - nlon:nrow, col].sum() / nlon
    mRh[-1, :, col] = rho[nrow - nlon:nrow, col].sum() / nlon
for i in range(1, nlat):
    mRl[i, :, :] = (bnd[i * nlon:i * nlon + nlon, :] + bnd[(i - 1) * nlon:i * nlon, :]) / 2
    mVp[i, :, :] = (v_p[i * nlon:i * nlon + nlon, :] + v_p[(i - 1) * nlon:i * nlon, :]) / 2
    mVs[i, :, :] = (v_s[i * nlon:i * nlon + nlon, :] + v_s[(i - 1) * nlon:i * nlon, :]) / 2
    mRh[i, :, :] = (rho[i * nlon:i * nlon + nlon, :] + rho[(i - 1) * nlon:i * nlon, :]) / 2

# revert south to north
mRl = np.flip(mRl, axis=0)
mVp = np.flip(mVp, axis=0)
mVs = np.flip(mVs, axis=0)
mRh = np.flip(mRh, axis=0)

# extend longitude from [-179.5, 179.5] to [-180., 180.]
def extend_lat(x):
    x_ext = np.zeros((nlat + 1, nlon + 2, nlayer))
    x_ext[:, 0, :] = (x[:, 0, :] + x[:, -1, :]) / 2
    x_ext[:, nlon + 1, :] = x_ext[:, 0, :]
    x_ext[:, 1:nlon + 1, :] = x[:, :, :]
    return x_ext
mRl = extend_lat(mRl)
mVp = extend_lat(mVp)
mVs = extend_lat(mVs)
mRh = extend_lat(mRh)

# convert to SI
mRl *= 1e3
mVp *= 1e3
mVs *= 1e3
mRh *= 1e3

# layers
if INCLUDE_ICE:
    INCLUDE_SEDIMENT = True
if INCLUDE_ICE:
    colSurf = 1
elif INCLUDE_SEDIMENT:
    colSurf = 2
else:
    colSurf = 5
colMoho = 8

# linear mapping to sphere
rdiff = (RADIUS_SURFACE - RADIUS_MOHO) / (mRl[:, :, colSurf] - mRl[:, :, colMoho])
elev = mRl.copy()
for i in range(nlayer):
    mRl[:, :, i] = rdiff * (elev[:, :, i] - elev[:, :, colMoho]) + RADIUS_MOHO

In [ ]:
# structured grid
grid_lat = np.arange(-90., 90.01, 1)
grid_lon = np.array([-180.] + list(np.arange(-179.5, 179.501, 1)) + [180.])
grid_r = np.linspace(RADIUS_MOHO, RADIUS_SURFACE, N_ELEM_LAYERS * 10)
data_vp = np.zeros((len(grid_lat), len(grid_lon), len(grid_r)))
data_vs = np.zeros((len(grid_lat), len(grid_lon), len(grid_r)))
data_rh = np.zeros((len(grid_lat), len(grid_lon), len(grid_r)))

def interp_2pnts(x, y, l, target):
    return (y[l] - y[l - 1]) / (x[l] - x[l - 1]) * (target - x[l - 1]) + y[l - 1]
    
for ilat in range(len(grid_lat)):
    for ilon in range(len(grid_lon)):
        rl_profile = mRl[ilat, ilon, :][colSurf:][::-1]
        vp_profile = mVp[ilat, ilon, :][colSurf:][::-1]
        vs_profile = mVs[ilat, ilon, :][colSurf:][::-1]
        rh_profile = mRh[ilat, ilon, :][colSurf:][::-1]
        # get rid of zero-thickness layers
        loc_non_zero = np.where(rh_profile > .1)[0]
        rl_profile = rl_profile[loc_non_zero]
        vp_profile = vp_profile[loc_non_zero]
        vs_profile = vs_profile[loc_non_zero]
        rh_profile = rh_profile[loc_non_zero]
        loc = np.searchsorted(rl_profile, grid_r, 'right')
        loc = np.minimum(loc, len(rl_profile) - 1)
        data_vp[ilat, ilon, :] = vp_profile[loc]
        data_vs[ilat, ilon, :] = vs_profile[loc]
        data_rh[ilat, ilon, :] = rh_profile[loc]

In [ ]:
# check
print('vp range:', data_vp.min(), data_vp.max())
print('vs range:', data_vs.min(), data_vs.max())
print('rho range:', data_rh.min(), data_rh.max())

fig, ax = plt.subplots(1, 3, dpi=200)
ax[0].imshow(data_vp[:, :, -1], origin='lower')
ax[0].set_title('vp on surface', fontsize=6)
ax[1].imshow(data_vs[:, :, -1], origin='lower')
ax[1].set_title('vs on surface', fontsize=6)
ax[2].imshow(data_rh[:, :, -1], origin='lower')
ax[2].set_title('rho on surface', fontsize=6)
for x in ax:
    x.axis('off')
plt.show()

fig, ax = plt.subplots(1, 3, dpi=200)
ax[0].imshow(data_vp[:, :, 0], origin='lower')
ax[0].set_title('vp on moho', fontsize=6)
ax[1].imshow(data_vs[:, :, 0], origin='lower')
ax[1].set_title('vs on moho', fontsize=6)
ax[2].imshow(data_rh[:, :, 0], origin='lower')
ax[2].set_title('rho on moho', fontsize=6)
for x in ax:
    x.axis('off')
plt.show()

In [ ]:
# write to file
nc = Dataset('input_with_3d_crust/crust1_volumetric.nc', 'w')
nc.createDimension('nlat', size=len(grid_lat))
nc.createDimension('nlon', size=len(grid_lon))
nc.createDimension('nr', size=len(grid_r))
nc.createVariable('latitude', float, dimensions=('nlat'))
nc['latitude'][:] = grid_lat
nc.createVariable('longitude', float, dimensions=('nlon'))
nc['longitude'][:] = grid_lon
nc.createVariable('radius', float, dimensions=('nr'))
nc['radius'][:] = grid_r
nc.createVariable('vp', float, dimensions=('nlat', 'nlon', 'nr'))
nc['vp'][:, :, :] = data_vp
nc.createVariable('vs', float, dimensions=('nlat', 'nlon', 'nr'))
nc['vs'][:, :, :] = data_vs
nc.createVariable('rho', float, dimensions=('nlat', 'nlon', 'nr'))
nc['rho'][:, :, :] = data_rh
nc.close()

## The geometric models

In [ ]:
# get undulation
undulation_surf = elev[:, :, colSurf]
undulation_moho = elev[:, :, colMoho] - RADIUS_MOHO + RADIUS_SURFACE

# smoothening
# if instability occurs, use a larger sigma smooth undulations more 
sigma = 2
undulation_surf_smooth = gaussian(undulation_surf, sigma=sigma)
undulation_moho_smooth = gaussian(undulation_moho, sigma=sigma)

# plot
fig, ax = plt.subplots(2, 2, dpi=200)
ax[0, 0].imshow(undulation_surf, origin='lower')
ax[0, 0].set_title('surface before smooth', fontsize=6)
ax[0, 1].imshow(undulation_moho, origin='lower')
ax[0, 1].set_title('moho before smooth', fontsize=6)
ax[1, 0].imshow(undulation_surf_smooth, origin='lower')
ax[1, 0].set_title('surface after smooth', fontsize=6)
ax[1, 1].imshow(undulation_moho_smooth, origin='lower')
ax[1, 1].set_title('moho after smooth', fontsize=6)
for xx in ax:
    for x in xx:
        x.axis('off')
plt.show()

In [ ]:
# write to file
nc = Dataset('input_with_3d_crust/crust1_geometric.nc', 'w')
nc.createDimension('nlat', size=len(grid_lat))
nc.createDimension('nlon', size=len(grid_lon))
nc.createVariable('latitude', float, dimensions=('nlat'))
nc['latitude'][:] = grid_lat
nc.createVariable('longitude', float, dimensions=('nlon'))
nc['longitude'][:] = grid_lon
nc.createVariable('undulation_surf', float, dimensions=('nlat', 'nlon'))
nc['undulation_surf'][:, :] = undulation_surf_smooth
nc.createVariable('undulation_moho', float, dimensions=('nlat', 'nlon'))
nc['undulation_moho'][:, :] = undulation_moho_smooth
nc.close()

## The ocean-load models

In [ ]:
# get ocean depth
ocean_depth = elev[:, :, 0] - elev[:, :, 1]

# plot
plt.figure(dpi=100)
plt.imshow(ocean_depth, origin='lower')
plt.axis('off')
plt.show()

In [ ]:
# write to file
nc = Dataset('input_with_3d_crust/crust1_oceanload.nc', 'w')
nc.createDimension('nlat', size=len(grid_lat))
nc.createDimension('nlon', size=len(grid_lon))
nc.createVariable('latitude', float, dimensions=('nlat'))
nc['latitude'][:] = grid_lat
nc.createVariable('longitude', float, dimensions=('nlon'))
nc['longitude'][:] = grid_lon
nc.createVariable('ocean_depth', float, dimensions=('nlat', 'nlon'))
nc['ocean_depth'][:, :] = ocean_depth
nc.close()

---